# Utilizando .csv em banco de dados SQL Server
> A tutorial de como utilizar pyodbc para salvar um arquivo csv no SQL Server

- toc: false
- badges: true
- comments: true
- categories: [csv, image]
- image: 

Para salvar um arquivo .csv utilizando python precisamos que na máquina local tenha instalado o gerenciador de banco de dados SQL Sever, caso não seja o caso faça o download da versão express (gratuita) no link: https://www.microsoft.com/pt-br/sql-server/sql-server-downloads.

Para utilizarmos python para acessar bancos de dados precisamos instalar o `pyodbc`, o qual possibilita a conexão com diversos tipos de banco de dados. Para instalar é só digitar no prompt de comando `pip install pyodbc`. Para maiores mais detalhes: https://pypi.org/project/pyodbc/.

## Carregando os pacotes do python

In [1]:
#carregando os pacotes necessários
import pyodbc
import pandas as pd
import csv

## Fazendo conexão com o banco de dados

Antes de mais nada precisamos conectar com o banco de dados que está instalado, o qual necessita dos seguintes dados:

- SERVER (nome do servidor);
- DATABASE (nome do seu banco de dados);
- USERNAME (nome de usuário do banco de dados);
- PASSWORD (senha do banco de dados);
- DRIVER (tipo do banco de dados).

Para estabelecer uma conexão utiliza-se o método `pyodbc.connect` da seguinte forma:

`pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)`

In [2]:
#hide
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=LAPTOP-93DTTS7R\SQLEXPRESS;DATABASE=SQL_DB_1;Trusted_Connection = yes')

## Criando cursor

Para manipularmos os dados de um banco de dados utilizando uma API (Application Programming Interface), necessita-se de fazermos utilizando um cursor. Os cursores possibilitam é um objeto que aponta para uma determinada linha dentro de um conjunto, o este que irá realizar todas as operações no banco de dados.

Para declarar iniciar um cursor utiliza-se o método `.cursor()`.

Obs.: Nunca deve-mos deixar de fechar o cursor após o uso.

In [3]:
cur = conn.cursor()

## Criando uma tabela no SQL Server

Com o cursor aberto, podemos utiliza-lo para realizarmos as consultas (*queries*) no banco de dados, da qual está excluindo a tabela no banco de dados com o nome `Socioeconomic_chicago` se ela existir.

Observe que podemos fazer todas as consultados nos banco de dados utilizando o método `execute()`, assim podemos escrever as `strings` que serão utilizada para esse fim.

In [4]:
cur.execute('DROP TABLE IF EXISTS Socioeconomic_chicago')

Segue o link para download do arquivo `.csv` utilizado:

https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

Como não haverá mais conflito com uma tabela igual, pode-se utilizar o comando `CREATE TABLE` para criar uma nova tabela.

In [5]:
cur.execute(''' CREATE TABLE Socioeconomic_chicago(
                Community_Area_Number INT PRIMARY KEY,
                COMMUNITY_AREA_NAME VARCHAR(50),
                PERCENT_OF_HOUSING_CROWDED FLOAT,
                PERCENT_HOUSEHOLDS_BELOW_POVERTY FLOAT,
                PERCENT_AGED_16_UNEMPLOYED FLOAT,
                PERCENT_AGED_25_WITHOUT_HIGH_SCHOOL_DIPLOMA FLOAT,
                PERCENT_AGED_UNDER_18_OR_OVER_64 FLOAT,
                PER_CAPITA_INCOME FLOAT,
                HARDSHIP_INDEX INT,

)
'''
)

## Salvando os dados no banco de dados

Declarar o caminho onde está localizado o arquivo em `.csv`.

`file = "C:/caminho/arquivo.csv"`

In [1]:
#hide
file = '''C:/Users/lucas/Documents/lucastiagooliveira/db/Socioeconomics_Chicago/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv'''

Neste passo será executado um loop para percorrer todas as linhas do arquivo `.csv` e gravar no banco de dados utilizando a *query* `INSERT INTO nome_tabela (coluna1, coluna2, coluna3, ...) VALUES (valor1, valor2, valor3, ...)`.

In [7]:
i = 0
with open(file) as csv_file:
    csv_reader =  csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        if i == 0:
            pass
        else:
            try:
                col1 = int(row[0])
                col2  = row[1]
                col3 = float(row[2])
                col4 = float(row[3])
                col5 = float(row[4])
                col6 = float(row[5])
                col7 = float(row[6])
                col8 = float(row[7])
                col9 = int(row[8])
            except:
                print('Values is broken, impossible to save in database. Code=', i)
            cur.execute(''' INSERT INTO Socioeconomic_chicago (Community_Area_Number, COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY, PERCENT_AGED_16_UNEMPLOYED,
                PERCENT_AGED_25_WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX)
                VALUES (?,?,?,?,?,?,?,?,?)

            ''',(col1, col2, col3, col4, col5, col6, col7, col8, col9))
        i += 1

Pronto tabela salva no bando de dados!

## Verificação todas as linhas foram salvas

Carregando todos os dados da tabela utilizando o `pandas`.

In [8]:
df = pd.read_sql_query('''SELECT * FROM Socioeconomic_chicago''', conn)


> Important: Feche a conexão com o banco de dados.

In [9]:
cur.close()

## Imprimindo a tabela carregada no banco de dados.

In [10]:
df

,Community_Area_Number,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16_UNEMPLOYED,PERCENT_AGED_25_WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939.0,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040.0,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787.0,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524.0,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123.0,6
...,...,...,...,...,...,...,...,...,...
73,74,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381.0,16
74,75,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149.0,30
75,76,O'Hare,3.6,15.4,7.1,10.9,30.3,25828.0,24
76,77,Edgewater,4.1,18.2,9.2,9.7,23.8,33385.0,19


Essa foi uma forma simples que encontrei para fazer essa tarefa, pois existem outras formas igualmente eficazes.

<sup>Powered by: Lucas Tiago</sup>